In [ ]:
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import rdFMCS
from matplotlib import colors
from rdkit.Chem.Draw import MolToImage
from rdkit.Chem.Draw import rdMolDraw2D

In [ ]:
def get_mol (smiles) :
    mol = Chem.MolFromSmiles (smiles)
    if mol is None:
        return None
    Chem.Kekulize (mol)
    return mol

In [ ]:
get_mol ('CCCCCCCCCC(=O)O')

In [ ]:
def find_matches_one(mol,submol) : 
    mols = [mol,submol]
    res=rdFMCS.FindMCS (mols)
    mcsp = Chem.MolFromSmarts(res.smartsString)
    matches = mol.GetSubstructMatches(mcsp)
    return matches

In [ ]:
def get_image(mol,atomset=None):    
    hcolor = colors.to_rgb('green')
    if atomset is not None:
        #highlight the atoms set while drawing the whole molecule.
        img = MolToImage(mol, size=(600, 600),fitImage=True, highlightAtoms=atomset,highlightColor=hcolor)
    else:
        img = MolToImage(mol, size=(400, 400),fitImage=True)
    return img

In [ ]:
peptide_smi = 'CCCCCCCCCC(=O)NC(Cc1c[nH]c2ccccc12)C(=O)NC(CC(N)=O)C(=O)NC(CC(=O)O)C(=O)NC4C(C)OC(=O)C(CC(=O)c3ccccc3(N))NC(=O)C(NC(=O)C(CO)NC(=O)CNC(=O)C(CC(=O)O)NC(=O)C(C)NC(=O)C(CC(=O)O)NC(=O)C(CCCN)NC(=O)CNC4(=O))C(C)CC(=O)O'
monomer_smi = 'CCCCCCCCCC(=O)O'

peptide_mol = get_mol(peptide_smi) #whole molecule
monomer_mol = get_mol(monomer_smi) #substructure molecule

In [ ]:
matches = find_matches_one(peptide_mol,monomer_mol)

atomset = list(matches[0])

In [ ]:
matches

In [ ]:
img = get_image(peptide_mol, atomset)
img #see the whole molecule with the substructure highlighted.

In [ ]:
img = get_image(monomer_mol)
img 

In [ ]:
"""Check https://www.rdkit.org/docs/GettingStartedInPython.html?highlight=maccs#:~:text=%3E%3E%3E%20colours%20%3D%20%5B(0.8%2C0.0%2C0.8)%2C(0.8%2C0.8%2C0,hit_bonds%2C%0A...%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20highlightBondColors%3Dbond_cols)"""

In [ ]:
>>> hit_ats = list(peptide_mol.GetSubstructMatch(monomer_mol))
>>> hit_bonds = []
>>> for bond in monomer_mol.GetBonds():
...    aid1 = hit_ats[bond.GetBeginAtomIdx()]
...    aid2 = hit_ats[bond.GetEndAtomIdx()]
...    hit_bonds.append(peptide_mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())
>>> d = rdMolDraw2D.MolDraw2DCairo(500, 500) 
>>> rdMolDraw2D.PrepareAndDrawMolecule(d,peptide_mol, highlightAtoms=hit_ats,
...                                    highlightBonds=hit_bonds)

>>> colours = [(0.8,0.0,0.8),(0.8,0.8,0),(0,0.8,0.8),(0,0,0.8)]
>>> atom_cols = {}
>>> for i, at in enumerate(hit_ats):
...     atom_cols[at] = colours[i%4]
>>> bond_cols = {}
>>> for i, bd in enumerate(hit_bonds):
...     bond_cols[bd] = colours[3 - i%4]
>>>
>>> d = rdMolDraw2D.MolDraw2DCairo(500, 500)
>>> rdMolDraw2D.PrepareAndDrawMolecule(d, mol, highlightAtoms=hit_ats,
...                                    highlightAtomColors=atom_cols,
...                                    highlightBonds=hit_bonds,
...                                    highlightBondColors=bond_cols)